In [ ]:
from utils import *
pandarallel.initialize(progress_bar=False,nb_workers=15)


In [ ]:
raw_data_path = '../data/daily_info/hospital_info.txt'

In [ ]:
def getName(line:str):
    return re.findall('\[(.+)\](.+)', line)[0]

def getAddress(line:str):
    return re.findall('医院地址：(.+)', line)[0]

def getOpened(line:str):
    return re.findall('开诊科室：(.+)', line)[0]

def getClosed(line:str):
    return re.findall('停诊科室：(.+)', line)[0]

def getOutpatient(line:str):
    line = line.replace('，','')
    return re.findall("门诊电话：([^急诊电话|总机]+)", line)[0]
    
def getEmergency(line:str):
    line = line.replace('，','')
    return re.findall('急诊电话：([^总机]+)', line)[0]

def getGeneral(line:str):
    line = line.replace('，','')
    return re.findall('总机：(.+)', line)[0]

hospital_info = pd.DataFrame(columns=['district', 'hospital', 'address', 'closed', 'opening', 'outpatient', 'emergency','General'])
with open(raw_data_path) as f:
    CNT = -1
    for line in f:
        if re.match('\[(.+)\](.+)', line):
            CNT+=1
            hospital_info.loc[CNT, 'district'], hospital_info.loc[CNT, 'hospital'] = getName(line)
        elif re.match('医院地址：(.+)', line):
            hospital_info.loc[CNT, 'address'] = getAddress(line)
        elif re.match('停诊科室：(.+)', line):
            hospital_info.loc[CNT, 'closed'] = getClosed(line)
        elif re.match('开诊科室：(.+)', line):
            hospital_info.loc[CNT, 'opening'] = getOpened(line)
            district, hospital, address, closed, opened = '','','','无',''

        if re.match("门诊电话：([^急诊电话|总机]+)", line):
            hospital_info.loc[CNT, 'outpatient'] = getOutpatient(line) 
        if re.match('.*急诊电话：([^总机]+)', line):
            hospital_info.loc[CNT, 'emergency'] = getEmergency(line) 
        if re.match('.*总机：(.+)', line):
            hospital_info.loc[CNT, 'General'] = getGeneral(line) 

                



In [ ]:
hospital_info[['lon', 'lat']] = hospital_info.parallel_apply(
    lambda x: getGPS('上海市'+x['district']+x['hospital']), axis=1, result_type='expand')

hospital_info[['lon_wgs', 'lat_wgs']] = hospital_info.parallel_apply(
    lambda x: bd09_to_wgs84(x['lon'], x['lat']), axis=1, result_type='expand')


In [ ]:
hospital_info = hospital_info.fillna('')
hospital_info.head()

In [ ]:
# 更新 estate_info_POI_category.json

out_geojson_path = '../data/CaseInfo_April/hospital_info.json'

features = []
for row in hospital_info.itertuples():
    address = getattr(row, "address")
    lon = getattr(row, "lon_wgs")
    lat = getattr(row, "lat_wgs")
    name = getattr(row, "hospital")
    district = getattr(row, "district")
    closed =  getattr(row, "closed")
    opening = getattr(row, "opening")
    outpatient = getattr(row, "outpatient")
    emergency = getattr(row, "emergency")
    General = getattr(row, "General")



    features.append({
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [lon, lat]
        },
        "properties": {
            '区县': district,
            '医院': name,
            '地址': address,
            '停诊科室': closed,
            '开诊科室': opening,
            '门诊电话': outpatient,
            '急诊电话': emergency,
            '总机': General


        }
    })

save_data = {"type": "FeatureCollection", "features": features}
with open(out_geojson_path, "w", encoding='utf-8') as f:
    f.write(json.dumps(save_data, ensure_ascii=False, indent=4, separators=(',', ':') ) )